# Process the data that is pulled by the OMOP query

In [1]:
# imports
import argparse
import os
import json
import pandas as pd
import numpy as np

/cluster/home/t125594uhn/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/cluster/home/t125594uhn/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/tmp/ipykernel_1147425/956546953.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Set the below paths to the locations of the OMOP tables

Set outpath to the path that output files should be written

In [13]:
# define the paths to the tables 
pats_path = 'test/pats.csv'
smoke_path = 'test/smok.csv'
inds_path = 'test/inds.csv'
dhd_path = 'test/dhd.csv'
events_path = 'test/events.csv'
labs_path = 'test/labs.csv'
meds_path = 'test/meds.csv'
deaths_path = 'test/deaths.csv'

outpath = 'test'

# load the tables
pats = pd.read_csv(pats_path)
smoke = pd.read_csv(smoke_path)
inds = pd.read_csv(inds_path)
dhd = pd.read_csv(dhd_path)
events = pd.read_csv(events_path)
labs = pd.read_csv(labs_path)
meds = pd.read_csv(meds_path)
deaths = pd.read_csv(deaths_path)


if not os.path.exists(outpath):
    os.makedirs(outpath)

Set below global variables as needed.
e.g. set STUDY_CUTOFF_DATE as the data pull date
e.g. set keywords, concept codes, icd codes etc to match what they are in your system

In [3]:
# global variables defining the codes that are needed to match different conditions, labs, etc (project_lists.py in the project code)

STUDY_CUTOFF_DATE = '2025-08-25'
CV_EVENT_GAP_DAYS = 180


TRANS_KEYWORDS = ['orthotropic liver transplant', 'heterotropic liver transplant', 'liver transplant']
TRANS_CONCEPT_LIST = [467458,2100972, 4067459]
TRANS_DICT = {'liver transplant': [467458,2100972,4067459]} # put any codes mapping to a liver transplant here
INV_TRANS = {y:x for x in TRANS_DICT.keys() for y in TRANS_DICT[x]}

METAB_CODES = ['K75.81','K76.0']
METAB_CONCEPT_CODES = [45533616,35208359]
METAB_DICT = {'K75.81' : [45533616],
              'K76.0' : [35208359]  
}
INV_METAB = {y:x for x in METAB_DICT.keys() for y in METAB_DICT[x]}


ALD_CODES = ['K70']
ALD_CONCEPT_CODES = [35208330,35208331,35208332,45562507,45552946,45605952,\
                    45538545,45586730,45552947]
ALD_DICT = {'K70' : [35208330,35208331,35208332,45562507,45552946,45605952,\
                    45538545,45586730,45552947]}
INV_ALD = {y:x for x in ALD_DICT.keys() for y in ALD_DICT[x]}


CANCER_CODES = ['C22']
CANCER_CONCEPT_CODES = [35206146,35206147,35206148,35206149,35206150,35206151,\
                        45585987,35206152]
CANCER_DICT = {'C22' : [35206146,35206147,35206148,35206149,35206150,35206151,\
                        35206152,45585987]}
INV_CANCER = {y:x for x in CANCER_DICT.keys() for y in CANCER_DICT[x]}


HEP_CODES = ["B15", "B16", "B17", "B18", "B19"]
HEP_CONCEPT_CODES = list(range(35205759,35205776))+\
                        [45576259,45605221,45552212,45566540,45581151]
HEP_DICT = {'B15' : [35205759,35205760],
            'B16' : [35205761,35205762,35205763,35205764],
            'B17' : [35205765,35205766,35205767,35205768,45576259,45605221],
            'B18' : [35205769,35205770,35205771,35205772,35205773],
            'B19' : [35205775,45552212,45566540,45581151,45547442 ]
}
INV_HEP = {y:x for x in HEP_DICT.keys() for y in HEP_DICT[x]}
              
                        
FULM_CODES = ["K72.0"]
FULM_CONCEPT_CODES = [45567324,45543338]
FULM_DICT = {'K72.0' : [45567324,45543338]}
INV_FULM = {y:x for x in FULM_DICT.keys() for y in FULM_DICT[x]}


IMMUNE_CODES = ["K75.4", "K74.3", "K83.01"]
IMMUNE_CONCEPT_CODES = [35208356,35208349] # <-- TODO: add PSC
IMMUNE_DICT = {'K75.4' : [35208356],
               'K74.3' : [35208349],
               'K83.01' : []} # <-- TODO: add concept code PSC
INV_IMMUNE = {y:x for x in IMMUNE_DICT.keys() for y in IMMUNE_DICT[x]}


RE_TX_CODES = ["Z94.4", "T86.4"] # <-- TODO: consider taking out Z94.4, "History of liver transplant"
RE_TX_CONCEPT_CODES = [45561100,45551553,45594968,45609395,45590132,35225408]
RE_TX_DICT = {'Z94.4' : [35225408],
              'T86.4' : [45561100,45551553,45594968,45609395,45590132]}
INV_RE_TX = {y:x for x in RE_TX_DICT.keys() for y in RE_TX_DICT[x]}


NONSMOKER_CODE = 40770157
SMOKER_CODES = [40764333,40768543] # <-- current or former smoker
SMOKER_CONCEPT_CODES = [42709996,762499,4310250,4298794,37395605,762498]
SMOKER_DICT = {"SMOKER" : [42709996,762499,4310250,4298794,37395605,762498]}
SMOKER_INV = {y:x for x in SMOKER_DICT.keys() for y in SMOKER_DICT[x]}

DM_CODES = ['E10', 'E11']
DM_CONCEPT_CODES = [1326492,1326493,35206878,35206879,35206881,35206882,37200148,37200166,37200167,37200170] + \
                    list(range(37200191,37200255))+list(range(45533017,45533024))+[45537962,45542736,45542737,45542738] + \
                    list(range(45547621,45547628)) + [45552379,45552382,45552385,45557113]+[45561949,45566731,45576439,45576443,\
                    45581350] + list(range(45581352,45581356)) + [45586139, 45586140, 45591027, 45591029, 45591031] + \
                    list(range(45595795,45595800)) + list(range(45600636,45600643)) + list(range(45605398,45605405))
DM_DICT = {'E10' : [35206878,35206879,37200148,37200166,37200167,37200170,45533017,455330178,45542736,45542737,\
                    45547621,45547622,45547624,45552379,45552382,45576439,45581350,45595795,45600636,\
                    45600637,45600638,45600640,45605398],
           'E11' : [1326492,1326493,35206881,35206882,]+[x for x in range(37200191,37200255)]+
                   [45533019,45533020,45533021,45533022,45533023,45537962,45542738,\
                    45547625,45547626,45547627,45552385,45557113,45561949,45566731,45576443,\
                    45581352,45581353,45581354,45581355,45586139,45586140,45591027, 5591029,45591031,\
                    45595797,45595798,45595799,45600641,45600642,45605401,45605402,45605403,45605404]}
INV_DM = {y:x for x in DM_DICT.keys() for y in DM_DICT[x]}
    

HTN_CODES = ['I10', 'I11', 'I12', 'I13', 'I15']
HTN_CONCEPT_CODES = list(range(35207668,35207679))
HTN_DICT = {'I10' : [35207668],
            'I11' : [35207669,35207670],
            'I12' : [35207671,35207672],
            'I13' : [35207673,35207674],
            'I15' : [35207675,35207676,35207677,35207678,35207679]}
INV_HTN = {y:x for x in HTN_DICT.keys() for y in HTN_DICT[x]}


LIP_CODES = ['E78.0', 'E78.1', 'E78.2', 'E78.5']
LIP_CONCEPT_CODES = list(range(35207060,35207063))+[35207065,37200312,37200313]
LIP_DICT = {'E78.0' : [35207060,37200312,37200313],
            'E78.1' : [35207061],
            'E78.2' : [35207062],
            'E78.5' : [35207065]}
INV_LIP = {y:x for x in LIP_DICT.keys() for y in LIP_DICT[x]}


CV_CODES = ['I48.0', 'I48.1', 'I48.2', 'I48.3', 'I48.4', 'I48.9', 'I21', 'I22', 'I25.2',\
            'I50', 'I46', 'G45.3','G45.9', 'I25.1','I25.4', 'I25.5', 'I25.6','I25.7','I25.8','I25.9', \
            'I63', 'I34', 'I35', 'I36', 'I37', 'I69.3', 'I42', 'I65', 'I66', 'I67.0'] # 'I39.0', 'I39.1', 'I39.2', 'I39.3', 'I39.4', # 'I47', 'I49',
CV_CONCEPT_CODES = [35207396,35207399,45562340,35207396,35207399,45562340,45572079,35207684,35207685,45576865,1326588,45533436,\
                    45605779,45572080,45557536,1326590,1326591,35207686,45605781,35207702,35207703,35207704,35207705,35207706,\
                    45586572,45557538,45538373,45596199,45596199,45548013,45567168,785999,45605788,45601024,45591456,37402491,\
                    45576866,45596197,45548010,45601027,45605784,45543167,45586574,45548012,45567167,45562344,45605787,35207755,\
                    35207756,35207757,35207758,35207759,35207760,35207761,35207762,35207763,35207764,45572091,45538383,35207779,\
                    35207781,35207782,35207783,786000,786001,786002,37402490,37402503,37402504,35207784,35207785,1569171,1569172,\
                    1569173,1553751,1553752,1553753,1553754,45576876,45572094,45562353,35207786,35207787,35207788,35207789,\
                    35207790,35207791,45591468,35207792,35207793,45586587,45543182,45576878,45567180,45601038,45548022,45533456,\
                    45562355,45533457,45591469,45586588,45567181,1326606,1326607,1326608,1326609,1326601,1326602,1326603,1326604,\
                    1326605,45601041,35207820,45533463,35207821,45567187,45581782,37200496,45562362,45605806,45552802,45538396,\
                    45591474,45562363,45576888,45552803,45601045,45552806,45601047,1595597,1595598,37200498,45596212,37200499,\
                    45576885,45601043,45562365,45543187,45567188,37200502,45557553,45576887,45596213,45586593,45562367,45586594,\
                    37200506,45576889,45543190,37200507,45596214,45548029,45548030,37200508,45581783,45601044,45552805,37200509,\
                    45605809,45586595,37200510,45538397,45586596,45557555,45586597,45581784,45596215,45586598,35207823,45552807,\
                    45601048,45557556,45601049,45601050,45557557,35207827,35207828,45601060,37200539,37200544,37200545,45548047,\
                    45538412,45557562,45533478,45538413,45548048,45586611,45591489,45543202,45533479,45572111,45557563,45557564,\
                    45548049,45605822,45596223]
CV_DICT = {'I48.0':[35207784],
           'I48.1':[35207785,1553751,1553752],
           'I48.2':[1569171,1553753,1553754],
           'I48.3':[1569172],
           'I48.4':[1569173],
           'I48.9':[45576876,45572094],
           'I47':[35207781,35207782,35207783,786000,786001,786002,37402490,37402503,37402504],
           'I49':[45562353,35207786,35207787,35207788,35207789,35207790,35207791,45591468],
           'I21':[45562340,45562340,45572079,35207684,35207685,45576865,1326588,45533436,45605779,\
                  45572080,45557536,1326590,1326591],
           'I22':[35207686,45605781],
           'I25.1':[45586572,45601024,45591456,37402491,45576866,45596197],
           'I25.2':[35207702],
           'I25.4':[45557538,45538373,],
           'I25.5':[35207704],
           'I25.6':[35207705],
           'I25.7':[45548010,45601027,45605784,45543167,45586574,45548012],
           'I25.8':[45596199,45548013,45567168,785999,45605788,45567167,45562344,45605787],
           'I25.9':[35207706],
           'I50':[35207792,35207793,45586587,45543182,45576878,45567180,45601038,45548022,45533456,\
                  45562355,45533457,45591469,45586588,45567181,1326606,1326607,1326608,1326609,1326601,\
                  1326602,1326603,1326604,1326605],
           'I46':[45572091,45538383,35207779],
           'G45.3':[35207396],
           'G45.9':[35207399],
           'I63':[45601041,35207820,45533463,35207821,45567187,45581782,37200496,45562362,45605806,45552802,45538396,\
                  45591474,45562363,45576888,45552803,45601045,45552806,45601047,1595597,1595598,37200498,45596212,37200499,\
                  45576885,45601043,45562365,45543187,45567188,37200502,45557553,45576887,45596213,45586593,45562367,45586594,\
                  37200506,45576889,45543190,37200507,45596214,45548029,45548030,37200508,45581783,45601044,45552805,37200509,\
                  45605809,45586595,37200510,45538397,45586596,45557555,45586597,45581784,45596215],
           'I65':[45586598,35207823,45552807,5601049,45601050],
           'I66':[45557557,35207827],
           'I67.0':[35207828],
           'I69.3':[45601060,37200539,37200544,37200545,45548047,45538412,45557562,45533478,45538413,45548048,45586611,45591489,\
                    45543202,45533479,45572111,45557563,45557564,45548049,45605822,45596223],
       #     'I39.0':[], # <-- I think all of these have been reclassified
       #     'I39.1':[],
       #     'I39.2':[],
       #     'I39.3':[], 
       #     'I39.4':[],
           'I34':[], # <-- TODO: add these
           'I35':[],
           'I36':[],
           'I37':[],
           'I42':[35207755,35207756,35207757,35207758,35207759,35207760,35207761,35207762,35207763,35207764]}
CV_INV = {y:x for x in CV_DICT.keys() for y in CV_DICT[x]}


CAD_CHRONIC_CODES = ['I25.1','I25.4','I25.6','I25.7','I25.8','I25.9']
ARYTHMIA_CHRONIC_CODES = ['I48.0', 'I48.1', 'I48.2','I48.91']
HEART_FAIL_CHRONIC_CODES = ['I25.5','I50','I42']
VALV_CHRONIC_CODES = ['I34','I35','I36','I37']
CEREBRO_CHRONIC_CODES = ['I69.3']


ARYTHMIA_CODES = ['I48.0', 'I48.1', 'I48.2', 'I48.3', 'I48.4', 'I48.9']#, 'I47', 'I49']
ACS_CODES = ['I21','I22', 'I25.2', 'I46']
CAD_CODES = ['I25.1','I25.4','I25.6','I25.7','I25.8','I25.9']
VALV_CODES = ['I34','I35','I36','I37']
CEREBRO_CODES = ['G45.3', 'G45.9','I63','I69.3', 'I65', 'I66', 'I67.0']
HF_CODES = ['I25.5','I50','I42']

LABS_DICT = {
    'CREATININE': [3016723],
    'ALP': [3035995],
    'ALT': [3006923],
    'CYCLO': [3010375],
    'AST': [3013721],
    'TAC': [3026250],
    'BMI': [3038553]
}
LABS_INV = {y:x for x in LABS_DICT.keys() for y in LABS_DICT[x]}


CREATININE_ID = 3016723
ALP_ID = 3035995
ALT_ID = 3006923
CYCLO_ID = 3010375
AST_ID = 3013721
TAC_ID = 3026250
BMI_ID = 3038553

LIP_LAB_IDS = {
    'LDL': [3028288,3028437],
    'HDL':[], # TODO: Do we have this?
    'TOTAL_CHOLESTEROL': [3027114],
    
}


MEDS_DICT = {'ANTI_HTN': [779445,964322,964324,964325,974858,978556,978557,1308221,1308250,1308251,1314006,1314008,1314009,\
                        1314581,1314614,1317675,1328689,1331312,1332419,1332497,1332500,1332525,1332526,1332527,1332528,\
                        1334459,1334460,1334494,1334535,1340161,1341268,1341270,1341302,1350490,1350521,1350552,1351558,\
                        1351559,1351583,1351587,1361519,1363057,1363058,1363059,1363060,1395060,19018811,19019236,19019238,\
                        19019239,19022947,19022948,19022949,19028935,19028936,19029027,19067686,19073093,19073094,19074671,\
                        19074672,19074673,19078080,19078101,19080128,19080129,19101573,19124265,19127430,19127432,19127433,\
                        19133212,19133558,19133562,19133566,19133570,19133574,19133578,19133582,19133585,19133587,19133614,\
                        19133621,19133622,35605001,35605003,40162864,40162867,40162871,40162875,40162878,40165757,40165762,\
                        40165767,40165773,40165785,40165789,40166824,40166826,40166828,40166830,40167087,40167091,40167196,\
                        40167202,40167213,40167218,40167849,40171499,40171510,40171516,40171547,40171550,40171553,40171556,\
                        40171559,40171562,40171849,40171852,40171863,40171884,40171905,40171917,40184184,40184187,40184217,\
                        40185276,40185280,40185304,40221243,40224172,40224175,40224178,42629595,42629596,42629597,42629598,\
                        43560163,46221722,46221724,46287342,46287346],
             'ANTI_PLATELET': [1112841,1112892,1112922,1113346,1322189,1331312,19046742,\
                               19059056,19065472,19066057,19073712,19075601,19076600,19076621,19076623,35605960,40163720,\
                               40163724,40241188,46287538],
             'STATIN': [1526476,1526480,1539407,1539411,1539462,1539463,1545959,1545996,1545997,1551927,1551929,1552015,\
                        19019115,19019116,19019117,19023487,19077244,19077245,19077497,19077498,19098474,19112569,19123592,\
                        19129329,40165245,40165246,40165253,40165257,40165261,40165262,40165638,40165642,40175390,40175394,\
                        40175400,40175404]
          }
MEDS_INV = {y:x for x in MEDS_DICT.keys() for y in MEDS_DICT[x]}


In [5]:
# process the cohort demographic info 

# 1. define the funcs

def process_pats(pats):
    ''' PATS table processing.
        columns:
            - person_id
            - birth_date -> convert to datetime
            - transplant_date -> convert to datetime. Take the last transplant per patient
            - sex -> convert to binary: M=0, F=1. Remove any other values
            - procedure_name -> check that it's in the list and drop columns that aren't      
        new colums:
            - age_at_trans -> calculate from birth_date and transplant_date
    '''
    pats['birth_date'] = pd.to_datetime(pats['birth_date'], format='mixed')
    pats['transplant_date'] = pd.to_datetime(pats['transplant_date'], format='mixed')
    
    # keep only the last transplant per patient
    pats = pats.sort_values('transplant_date').groupby('person_id').tail(1)
    
    pats['sex'] = pats['sex'].apply(lambda x: 0 if x == 'Male' else 1 if x == 'Female' else np.nan)
        
    # drop pats with sex=nan and print warning
    missing_sex = pats.loc[pats['sex'].isna(), 'person_id'].values.tolist()
    if(len(missing_sex) > 0):
        print(f"Dropping {len(missing_sex)} patients with missing sex values:")
        print(missing_sex)
    pats = pats[pats['sex'].notna()]
    
    # drop pats with procedure_name not in list and print warning
    nontx_pats = pats.loc[~(pats['procedure_name'].str.contains('liver transplant', case=False)), 'person_id'].values.tolist()
    if len(nontx_pats) > 0:
        print(f"Dropping {len(nontx_pats)} patients with non-transplant procedure names:")
        print(nontx_pats)
    pats = pats[pats['procedure_name'].str.contains('liver transplant', case=False)]
    pats = pats.drop(columns=['procedure_name'])
    
    pats['age_at_tx'] = ((pats['transplant_date'] - pats['birth_date']).dt.days / 365.25).round(2)
    pats = pats.drop(columns=['birth_date'])
        
    # drop pats < 18 at tx
    if len(pats[pats['age_at_tx'] < 18]) > 0:
        print(f"Dropping {len(pats[pats['age_at_tx'] < 18])} patients < 18 at transplant:")
    pats = pats[pats['age_at_tx'] >= 18]
    
    return pats


def process_deaths(cohort, deaths):
    ''' DEATHS_CTE table processing.
        columns:
            - person_id
            - death_date -> convert to datetime
        new colums:
            - CENSOR_DATE -> date of death or study end date for those with no death date
            
    '''
    deaths['death_date'] = pd.to_datetime(deaths['death_date'], format='mixed')
    # merge with cohort on person_id
    cohort = pd.merge(cohort, deaths[['person_id', 'death_date']], on='person_id', how='left')
    # set the censor date to the death date or study end date
    cohort['CENSOR_DATE'] = cohort['death_date'].fillna(pd.to_datetime(STUDY_CUTOFF_DATE))
    # drop the death date column
    cohort = cohort.drop(columns=['death_date'])
    
    # drop patients censored less than 1.25 years post-transplant
    censored = cohort[cohort['CENSOR_DATE'] < (cohort['transplant_date'] + pd.DateOffset(years=1, months=3))]
    if len(censored) > 0:
        print(f"Dropping {len(censored)} patients with censor date < 1.25 years post-transplant:")
        print(censored['person_id'].values.tolist())

    cohort = cohort[cohort['CENSOR_DATE'] >= (cohort['transplant_date'] + pd.DateOffset(years=1, months=3))]

    return cohort



# 2. run the funcs

cohort = process_pats(pats)
cohort = process_deaths(cohort, deaths)

In [8]:
# process smoking info


# funcs
def process_smoke(cohort, smoke):
    ''' SMOKING_CTE table processing.
        columns:
            - person_id
            - smoking_status -> codes for current, former, never.
            - observation_date
        new columns:
            - SMOKER -> binary: 0 if never, 1 else.
    '''
    smoker_ids = smoke.loc[smoke['smoking_status'].isin(SMOKER_CONCEPT_CODES), 'person_id'].values.tolist()
    cohort['SMOKER'] = cohort['person_id'].apply(lambda x: 1 if x in smoker_ids else 0)
    
    return cohort

# run

cohort = process_smoke(cohort, smoke)

In [10]:
# process transplant indications

# funcs
def process_inds(cohort, inds, pats):
    ''' INDICATION_CTE table processing.
        columns:
            - person_id
            - diagnosis_date -> convert to date, keep only the one at the final transplant
            - diagnosis
            - icd10_code -> map from codes to indication columns
            
        new columns:
            - METAB -> metabolic syndrome as indication
            - ALD -> alcoholic liver disease as indication
            - CANCER -> HCC as indication
            - HEP -> hepatitis as indication
            - FULM -> fulminant liver failure as indication
            - IMMUNE -> autoimmune liver disease as indication
            - RE_TX -> re-transplant as indication
    '''
    inds['diagnosis_date'] = pd.to_datetime(inds['diagnosis_date'], format='mixed')
    
    # the code can be a prefix of the full code, so we need to check for that
    inds['METAB'] = inds['icd10_code'].apply(lambda x: 1 if any([x.startswith(c) for c in METAB_CODES]) else 0)    
    inds['ALD'] = inds['icd10_code'].apply(lambda x: 1 if any([x.startswith(c) for c in ALD_CODES]) else 0)
    inds['CANCER'] = inds['icd10_code'].apply(lambda x: 1 if any([x.startswith(c) for c in CANCER_CODES]) else 0)
    inds['HEP'] = inds['icd10_code'].apply(lambda x: 1 if any([x.startswith(c) for c in HEP_CODES]) else 0)
    inds['FULM'] = inds['icd10_code'].apply(lambda x: 1 if any([x.startswith(c) for c in FULM_CODES]) else 0)
    inds['IMMUNE'] = inds['icd10_code'].apply(lambda x: 1 if any([x.startswith(c) for c in IMMUNE_CODES]) else 0)
    ## do RE_TX separately using the pats table !
    ## inds['RE_TX'] = inds['icd10_code'].apply(lambda x: 1 if any([x.startswith(c) for c in RE_TX_CODES]) else 0)
    
    # merge all the rows of each patient into a single row
    inds = inds.groupby('person_id').agg({'METAB':'max', 'ALD':'max', 'CANCER':'max', 'HEP':'max', 'FULM':'max', 'IMMUNE':'max', 'diagnosis_date':'min'}).reset_index()    
    
    ## C-S cohort has unexpectedly very high number of FULM - this is unlikely,
    ## probably these were coded with K72.0 due to some difference in coding practice
    ## Exclude any of these that are also one of the other conditions
    inds['FULM'] = (inds['FULM'] & ~(inds['METAB'] | inds['ALD'] | inds['CANCER'] | inds['HEP'] | inds['IMMUNE'])).astype(int)
        
    
    cohort = pd.merge(cohort, inds, on='person_id', how='left')
    # TODO: Figure out best way to do this date filtering.
    # pre_tx_timedelta = (cohort['transplant_date'] - cohort['diagnosis_date']).dt.days
    # # cohort = cohort.loc[((cohort['RE_TX']==1)&(pre_tx_timedelta>1))|
    # #                     ((pre_tx_timedelta <= 365)&(pre_tx_timedelta >= 0))]
    # cohort = cohort.loc[(pre_tx_timedelta >= 0)]
        
    cohort = cohort.drop(columns = ['diagnosis_date'])
    
    pats['birth_date'] = pd.to_datetime(pats['birth_date'], format='mixed')
    pats['transplant_date'] = pd.to_datetime(pats['transplant_date'], format='mixed')
    # keep only the last transplant per patient
    pats = pats.sort_values(['person_id','transplant_date'])
    def had_prior_tx(subdf):
        last_date = subdf['transplant_date'].dt.normalize().iloc[-1]
        earlier_dates = subdf['transplant_date'].dt.normalize() < last_date  # strictly earlier
        return int(earlier_dates.any())

    indicator = (
        pats.groupby('person_id')
        .apply(had_prior_tx)
        .rename('RE_TX')
    )
    
    cohort = cohort.merge(indicator, left_on='person_id', how='left', right_index=True)
    
    return cohort

# run
cohort = process_inds(cohort, inds, pats)


In [11]:
# process health statuses

# funcs

# Helper function to apply conditions across time
def mark_condition(cohort, dhd, condition_name, code_list):
    end_date = pd.to_datetime(STUDY_CUTOFF_DATE)
    max_years = int((end_date - cohort['transplant_date'].min()).days / 365.25)
    condition_df = dhd[dhd['icd10_code'].str.startswith(tuple(code_list))]
    merged = condition_df.merge(cohort[['person_id', 'transplant_date']], on='person_id', how='left')
    merged['years_since_tx'] = ((merged['diagnosis_date'] - merged['transplant_date']).dt.days / 365.25)

    for i in range(1, max_years + 1):
        hits = merged.loc[merged['years_since_tx'] < (i+0.25), 'person_id'].unique()
        cohort[f'{condition_name}_{i}'] = cohort['person_id'].isin(hits).astype(int)
    return cohort


def process_dhd(cohort, dhd):
    ''' DHD_CTE table processing.
        columns:
            - person_id
            - diagnosis_date -> convert to date, keep only the one at the final transplant
            - diagnosis
            - icd10_code -> map from codes to indication columns
            
        new columns:
            - DIABETES_<yr> -> diabetes as diagnosis
            - HYPERTENSION_<yr> -> hypertension as diagnosis
            - DYSLIPIDEMIA_<yr> -> dyslipidemia as diagnosis
            
            
        NOTE: we construct the columns for each year from the transplant date.
        NOTE: Years are offset by 3 months, basically we start counting from 3 months post-tx.         
    '''
    # for each patient create columns DM_1, DM_2,... HTN_1... LIP_1... 
    # up to now from 3 months after the transplant
    dhd['diagnosis_date'] = pd.to_datetime(dhd['diagnosis_date'], format='mixed')

    # Apply for each condition
    cohort = mark_condition(cohort, dhd, 'DM', DM_CODES)
    cohort = mark_condition(cohort, dhd, 'HTN', HTN_CODES)
    cohort = mark_condition(cohort, dhd, 'LIP', LIP_CODES)

    return cohort


# run
cohort = process_dhd(cohort, dhd)

In [14]:
# process CV events

# funcs
def match_chronic(df,codes):    
    is_match = df['icd10_code'].str.startswith(tuple(codes))
    # Get first match per patient
    first_match = df[is_match].groupby('person_id', as_index=False).first()
    # Get all non-matching rows
    non_match = df[~is_match]
    # Combine them
    result = pd.concat([non_match, first_match], ignore_index=True).sort_values(['person_id', 'diagnosis_date'])
    return result


def group_events(df, gap=30):
    df = df.sort_values(by = ['person_id','diagnosis_date']).reset_index(drop=True)
    df_to_collapse = df[df["icd10_code"].isin(CV_CODES)].copy()
   # df_other = df[~df["icd10_code"].isin(CV_CODES)].copy() # this should be empty

    # compute gap
    df_to_collapse["prev_date"] = df_to_collapse.groupby(["person_id","icd10_code"])["diagnosis_date"].shift()
    df_to_collapse["days_since_prev"] = (df_to_collapse["diagnosis_date"] - df_to_collapse["prev_date"]).dt.days

    # new cluster whenever first event or gap exceeded
    df_to_collapse["new_cluster"] = (df_to_collapse["days_since_prev"].isna()) | (df_to_collapse["days_since_prev"] > gap)

    # cluster id
    df_to_collapse["cluster_id"] = df_to_collapse.groupby(["person_id","icd10_code"])["new_cluster"].cumsum()

    # now: keep *first event in each cluster* only
    collapsed = df_to_collapse.groupby(["person_id","icd10_code","cluster_id"]).first().reset_index(drop=True)

    # combine back with unaffected events
   # result = pd.concat([collapsed[["person_id","icd10_code","diagnosis_date"]], df_other], ignore_index=True)
    result = collapsed.sort_values(["person_id","diagnosis_date"]).reset_index(drop=True)
    
    return result


def process_events(cohort, events):
    ''' CV_EVENTS_CTE table processing.
        columns:
            - person_id
            - diagnosis_date
            - diagnosis 
            - icd10_code 
            
        new columns:
            - CV_HISTORY_<yr> -> past cardiovascular event as diagnosis
            - MONTHS_TO_EVENT_<yr> -> months to event for each year
    '''
    events['diagnosis_date'] = pd.to_datetime(events['diagnosis_date'], format='mixed')
    
    # drop anything here that is not coded as a CV event
    events = events[events['icd10_code'].str.startswith(tuple(CV_CODES))]
    events = events[events['person_id'].isin(cohort['person_id'])]
    
    # try to group repeated event codes into a single event - window = 7 days
    events = group_events(events, CV_EVENT_GAP_DAYS)
    
    cohort = mark_condition(cohort, events, 'CV_HISTORY', CV_CODES)
    
    # now we need to calculate the time to next event for each year
    merged = events.merge(cohort[['person_id', 'transplant_date']], on='person_id', how='left')

    # Get max duration
    max_years = int((pd.to_datetime(STUDY_CUTOFF_DATE) - cohort['transplant_date'].min()).days / 365.25)

    # Sort for quick lookup
    merged = merged.sort_values(by=['person_id', 'diagnosis_date'])
    
    # chronic events are counted as events only the first time    
    merged = match_chronic(merged,CAD_CHRONIC_CODES) 
    merged = match_chronic(merged,ARYTHMIA_CHRONIC_CODES)
    merged = match_chronic(merged,VALV_CHRONIC_CODES)
    merged = match_chronic(merged,HEART_FAIL_CHRONIC_CODES)
    merged = match_chronic(merged,CEREBRO_CHRONIC_CODES)


    # Loop through each follow-up year
    for i in range(1, max_years + 1):
        colname = f'MONTHS_TO_EVENT_{i}'
        cohort[colname] = np.nan

        for idx, row in cohort.iterrows():
            pid = row['person_id']
            anchor_date = row['transplant_date'] + pd.DateOffset(years=i,months=3)

            # Get all  diagnoses for this patient after the anchor date
            future_df = merged[(merged['person_id'] == pid) & (merged['diagnosis_date'] > anchor_date)]

            if not future_df.empty:
                next_event_date = future_df['diagnosis_date'].iloc[0]
                time_to_event = (next_event_date - anchor_date).days / 30.4
                cohort.at[idx, colname] = time_to_event
        cohort[colname] = cohort[colname].round()
                
    return cohort, merged

# run
cohort, processed_events = process_events(cohort, events)


def get_cv_event_debugging_info(cohort, events):
    events['diagnosis_date'] = pd.to_datetime(events['diagnosis_date'], format='mixed')
    
    # drop anything here that is not coded as a CV event
    events = events[events['icd10_code'].str.startswith(tuple(CV_CODES))]
    events = events[events['person_id'].isin(cohort['person_id'])]
    
    # print off the following info:
    # number of patients for which any given icd10_code occurs
    # for the top ones of those: 
    #   # that were more that 1 year pre-transplant
    #   # that were in the 1 year pre-transplat
    #   # that were in the 1.25 years post-transplant
    #   # that were more than 1.25 years post-transplant
    
    # get the unique icd10_codes per patient
    code_stats = {}
    for code in events['icd10_code'].unique():
        code_df = events[events['icd10_code'] == code]
        unique_pats = code_df['person_id'].nunique()
        pre_1yr = 0
        pre_tx = 0
        post_1_25yr = 0
        post_1_25yr_plus = 0
        
        for idx, row in code_df.iterrows():
            pat_tx_date = cohort.loc[cohort['person_id'] == row['person_id'], 'transplant_date'].values[0]
            delta_days = (row['diagnosis_date'] - pat_tx_date).days
            
            if delta_days < -365:
                pre_1yr += 1
            elif -365 <= delta_days < 0:
                pre_tx += 1
            elif 0 <= delta_days <= 456.5:
                post_1_25yr += 1
            else:
                post_1_25yr_plus += 1
        code_stats[code] = {'unique_patients': unique_pats,
                            'pre_1yr': pre_1yr,
                            'pre_tx': pre_tx,
                            'post_1_25yr': post_1_25yr,
                            'post_1_25yr_plus': post_1_25yr_plus}
        pre_1yr_df = code_df[code_df['diagnosis_date'] < (pat_tx_date - pd.DateOffset(years=1))]
        pre_tx_df = code_df[(code_df['diagnosis_date'] >= (pat_tx_date - pd.DateOffset(years=1))) & (code_df['diagnosis_date'] < pat_tx_date)]
        post_1_25yr_df = code_df[(code_df['diagnosis_date'] >= pat_tx_date) & (code_df['diagnosis_date'] <= (pat_tx_date + pd.DateOffset(years=1, months=3)))]
        post_1_25yr_plus_df = code_df[code_df['diagnosis_date'] > (pat_tx_date + pd.DateOffset(years=1, months=3))]
        code_stats[code]['pre_1yr_unique'] = pre_1yr_df['person_id'].nunique()
        code_stats[code]['pre_tx_unique'] = pre_tx_df['person_id'].nunique()
        code_stats[code]['post_1_25yr_unique'] = post_1_25yr_df['person_id'].nunique()
        code_stats[code]['post_1_25yr_plus_unique'] = post_1_25yr_plus_df['person_id'].nunique()
        
        
    # sort by unique patients
    sorted_stats = dict(sorted(code_stats.items(), key=lambda item: item[1]['unique_patients'], reverse=True))
    print("CV Event ICD10 Code Stats:")
    for code, stats in sorted_stats.items():
        print(f"Code: {code}, Unique Patients: {stats['unique_patients']}, Pre-1yr: {stats['pre_1yr']}, Pre-1yr unique patients: {stats['pre_1yr_unique']}, \
                Pre-Tx: {stats['pre_tx']}, Pre-Tx unique patients: {stats['pre_tx_unique']} Post-1.25yr: {stats['post_1_25yr']}, Post-1.25yr unique patients: {stats['post_1_25yr_unique']}, \
                Post-1.25yr+: {stats['post_1_25yr_plus']}, Post-1.25yr+ unique patients: {stats['post_1_25yr_plus_unique']}")
sorted_stats_df = get_cv_event_debugging_info(cohort, events)
sorted_stats_df.to_csv(os.path.join(outpath, 'cv_event_debugging_stats.csv'), index=True)

/tmp/ipykernel_1147425/3249410689.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cohort[colname] = np.nan
/tmp/ipykernel_1147425/3249410689.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cohort[colname] = np.nan
/tmp/ipykernel_1147425/3249410689.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.

In [ ]:
# process labs

# funcs
def process_labs(cohort, labs):
    ''' LABS_CTE table processing.
        columns:
            - person_id
            - measurement_date
            - test_name 
            - test_code
            - test_value
            - test_unit
            
        new columns:
            - <lab>_<yr> -> lab value for each lab for each year
        NOTE: forward fill labs
        NOTE: labs we use: ALT, ALP, AST, BMI, CREATININE, CYCLO, TAC
    '''
    # filter out any labs from before 3 months post-tx
    labs['measurement_date'] = pd.to_datetime(labs['measurement_date'], format='mixed')
    lab_df = labs.merge(cohort[['person_id', 'transplant_date']], on='person_id', how='left')
    labs = lab_df[lab_df['measurement_date'] >= (lab_df['transplant_date'] + pd.DateOffset(months=3))].copy()
    labs = labs.sort_values(['person_id', 'measurement_date'])
    lab_cols = ['ALT', 'ALP', 'AST', 'BMI', 'CREATININE', 'CYCLO', 'TAC']
    
    max_years = int((pd.to_datetime(STUDY_CUTOFF_DATE) - cohort['transplant_date'].min()).days / 365.25)
    new_cols_df = pd.DataFrame(np.nan,index=cohort.index, columns=[f'{lab}_{i}' for lab in lab_cols for i in range(1, max_years + 1)])
    cohort = pd.concat([cohort, new_cols_df], axis=1)
    for lab in lab_cols:
        lab_subset = labs[labs['test_code'].isin(LABS_DICT[lab])].copy()
        for i in range(1, max_years + 1):
            col_name = f'{lab}_{i}'

            # Define the window of interest per patient
            for idx, row in cohort.iterrows():
                pid = row['person_id']
                anchor_date = row['transplant_date'] + pd.DateOffset(years=i, months=3)

                # Filter lab values before (or at) anchor date
                labs_for_patient = lab_subset[lab_subset['person_id'] == pid]
                labs_before = labs_for_patient[labs_for_patient['measurement_date'] <= anchor_date]

                if not labs_before.empty:
                    most_recent = labs_before.sort_values('measurement_date', ascending=False).iloc[0]['test_value']
                    cohort.at[idx, col_name] = most_recent
    for i in range(1, max_years + 1):
        cohort[f'CREATININE_{i}'] = cohort[f'CREATININE_{i}']*88.42            

    # adjust tac and cyclo:
    # if *any* tac values for a patient across all years, set all their cyclo to 0.
    # if no tac values for a patient, and any cyclo values, set all tac to 0.
    
    tac_cols = [f'TAC_{i}' for i in range(1, max_years + 1)]
    cyclo_cols = [f'CYCLO_{i}' for i in range(1, max_years + 1)]
    for idx, row in cohort.iterrows():
        pid = row['person_id']
        tac_values = [row[col] for col in tac_cols]
        cyclo_values = [row[col] for col in cyclo_cols]
        
        if any([(x > 0)for x in tac_values]):
            # set all cyclo values to 0
            for col in cyclo_cols:
                cohort.at[idx, col] = 0
        elif not any([(x > 0) for x in tac_values]) and any([(x > 0)for x in cyclo_values]):
            # set all tac values to 0
            for col in tac_cols:
                cohort.at[idx, col] = 0
                
    # MODIFICATION FOR UCSF COHORT: patients missing all labs
    
    lab_check_cols = [f'{lab}_{i}' for lab in lab_cols for i in range(1, max_years + 1)]
    
    # nan or 0 in all lab check cols
    missing_all = cohort[(cohort[lab_check_cols].isna() | (cohort[lab_check_cols]==0)).all(axis=1)]
    print(f"{len(missing_all)} patients missing all lab values after 3 months post-transplant.")
    # missing all after 1 year
    missing_after_1yr = cohort[(cohort[[col for col in lab_check_cols if not col.endswith('_1')]].isna() | (cohort[[col for col in lab_check_cols if not col.endswith('_1')]]==0)).all(axis=1)]
    print(f"{len(missing_after_1yr)} patients missing all lab values after 1 year, 3 months post-transplant.")
    print("Dropping patients missing all lab values after 1 year:")
    print(missing_after_1yr['person_id'].values.tolist())
    print("Range of transplant dates for these patients:")
    print(missing_after_1yr['transplant_date'].describe())
    
    cohort = cohort[~cohort['person_id'].isin(missing_after_1yr['person_id'])]
    
    return cohort

# run
cohort = process_labs(cohort, labs)

In [18]:
# process medications

# funcs
def process_meds(cohort, meds):
    ''' MEDS_CTE table processing.
        columns:
            - person_id
            - start_date
            - end_date
            - medication_name
            - medication_code
            - dosage
            
        new columns:
            - ANTI_HTN_<yr>
            - ANTI_PLATELET_<yr>
            - STATIN_<yr>
        NOTE: forward fill meds
    '''
    meds['start_date'] = pd.to_datetime(meds['start_date'], format='mixed')
    meds = meds.sort_values(['person_id', 'start_date'])
    
    med_cols = ['ANTI_HTN', 'ANTI_PLATELET', 'STATIN']
    
    max_years = int((pd.to_datetime(STUDY_CUTOFF_DATE) - cohort['transplant_date'].min()).days / 365.25)

    for med in med_cols:
        med_subset = meds[meds['medication_code'].isin(MEDS_DICT[med])].copy()
        # keep only the first medication per patient
        med_subset = med_subset.groupby('person_id').first().reset_index()
        cohort = cohort.merge(med_subset[['person_id', 'start_date']], on='person_id', how='left')
        for i in range(1, max_years + 1):
            col_name = f'{med}_{i}'
            cohort[col_name] = 0
            cohort[col_name] = (cohort['start_date'] <= (cohort['transplant_date'] + pd.DateOffset(years=i, months=3))).astype(int)
            # Drop the start_date column
        cohort = cohort.drop(columns=['start_date'])
        
    return cohort

# run
cohort = process_meds(cohort, meds)

In [19]:
# update the health statuses using medication and lab info

# funcs
def add_dhd(cohort, labs):
    ''' Additional updates to the diseases:
        - Anyone on ANTI_HTN meds has HTN, anyone on STATIN has LIP
        - Any LDL > 4.1 or tryglycerides > 2.3 or Total cholesterol > 5.2 inidicative of LIP 
    '''
    
    max_yrs = int((pd.to_datetime(STUDY_CUTOFF_DATE) - cohort['transplant_date'].min()).days / 365.25)
    for i in range(1, max_yrs + 1):
        cohort[f'HTN_{i}'] |= cohort[f'ANTI_HTN_{i}']
        cohort[f'LIP_{i}'] |= cohort[f'STATIN_{i}']
        
    labs_subset = labs[labs['test_name'].isin([x for k in LIP_LAB_IDS.keys() for x in LIP_LAB_IDS[k]])].copy()
    
    # TODO: finish this with triglycerides and total cholesterol
    labs_subset = labs_subset[(labs_subset['test_name'].isin(LIP_LAB_IDS['LDL'])&\
                                labs_subset['test_value'] > 4.1)]
    labs_subset = labs_subset.sort_values(['person_id', 'measurement_date'])
    # take the first irregular lab for each patient
    labs_subset = labs_subset.groupby('person_id').first().reset_index()
    # merge with cohort
    cohort = cohort.merge(labs_subset[['person_id', 'measurement_date']], on='person_id', how='left')
    # for each year, if the lab is before the anchor date, set LIP_<yr> to 1
    for i in range(1, max_yrs + 1):
        cohort[f'LIP_{i}'] |= (cohort['measurement_date'] <= (cohort['transplant_date'] + pd.DateOffset(years=i,months=3))).astype(int)
    cohort.drop(columns=['measurement_date'], inplace=True)
    
    return cohort

# run
cohort = add_dhd(cohort, labs)

In [22]:
# Save cohort information

# funcs
def get_cohort_info(cohort, processed_events, outdir):
    ''' Dump demographic info and stats on the cohort to a json file.
    '''
    
    demo_dict = {}
    demo_dict['Females'] = cohort['sex'].sum()
    demo_dict['Males'] = len(cohort) - demo_dict['Females']
    demo_dict['Age'] = {'Median': cohort['age_at_tx'].median(), 'Lower': cohort['age_at_tx'].quantile(0.25),
                        'Upper': cohort['age_at_tx'].quantile(0.75)}
    demo_dict['Current, ex-smokers'] = cohort['SMOKER'].sum()
    demo_dict['Indications'] = {x:cohort[x].sum() for x in ['METAB', 'ALD', 'CANCER', 'HEP', 'FULM', 'IMMUNE', 'RE_TX']}
        
    bin_varying = ['DM', 'HTN', 'LIP', 'CV_HISTORY', 'ANTI_HTN', 'ANTI_PLATELET', 'STATIN']
    max_years = int((pd.to_datetime(STUDY_CUTOFF_DATE) - cohort['transplant_date'].min()).days / 365.25)
    for v in bin_varying:
        demo_dict[v] = {'First':cohort[f'{v}_1'].sum(), 'Last':cohort[f'{v}_{max_years}'].sum()}
    
    labs = ['ALT', 'ALP', 'AST', 'BMI', 'CREATININE', 'CYCLO', 'TAC']
    for l in labs:
        demo_dict[l] = {}
        demo_dict[l]['Missing, final'] = cohort[f'{l}_{max_years}'].isna().sum()
        demo_dict[l]['Median, final'] = cohort[f'{l}_{max_years}'].median()
        demo_dict[l]['Lower, final'] = cohort[f'{l}_{max_years}'].quantile(0.25)        
        demo_dict[l]['Upper, final'] = cohort[f'{l}_{max_years}'].quantile(0.75)
    # for tac and cyclosporine median, upper, lower should be of the non-zero. Missing should include zeros:
    for l in ['TAC', 'CYCLO']:
        demo_dict[l]['Median, final'] = cohort[f'{l}_{max_years}'][cohort[f'{l}_{max_years}'] > 0].median()
        demo_dict[l]['Lower, final'] = cohort[f'{l}_{max_years}'][cohort[f'{l}_{max_years}'] > 0].quantile(0.25)        
        demo_dict[l]['Upper, final'] = cohort[f'{l}_{max_years}'][cohort[f'{l}_{max_years}'] > 0].quantile(0.75)
        demo_dict[l]['Missing, final'] = cohort[f'{l}_{max_years}'].isna().sum() + cohort[f'{l}_{max_years}'][cohort[f'{l}_{max_years}'] == 0].sum()
    
    # number of first events for a patient, median, upper, and lower MONTHS_TO_EVENT - for first event
    # number of rows with a non-null value in one of the MONTHS_TO_EVENT columns
    # event_cols = [f'MONTHS_TO_EVENT_{i}' for i in range(1, max_years + 1)]
    # events_df = cohort[event_cols]
    
    # first_event_times = events_df.apply(lambda row: row[row.notna()].iloc[0] if row.notna().any() else np.nan, axis=1)
    # first_event_times = first_event_times.dropna()
    first_event_times = cohort['MONTHS_TO_EVENT_1'].dropna()
    median = first_event_times.median()
    lower_q = first_event_times.quantile(0.25)
    upper_q = first_event_times.quantile(0.75)
    demo_dict['CV_EVENTS'] = {}
    demo_dict['CV_EVENTS']['First'] = {'N':len(first_event_times),
                                     'Median':median,
                                     'Lower':lower_q, 'Upper':upper_q}
    processed_events = processed_events[processed_events['diagnosis_date'] >= (processed_events['transplant_date'] + pd.DateOffset(months=15))]
    processed_events = processed_events[processed_events['person_id'].isin(cohort['person_id'])]
    # times between events for the same patient
    times_df = processed_events[['person_id','diagnosis_date','transplant_date']].copy()
    times_df = times_df.drop_duplicates(subset=['person_id','diagnosis_date'])
    times_df = times_df.sort_values(by=['person_id','diagnosis_date']).reset_index(drop=True)
    times_df["months_since_prev"] = times_df.groupby("person_id")["diagnosis_date"].diff().dt.days / 30.4
    times_df["months_since_prev"] = times_df["months_since_prev"].fillna((((times_df["diagnosis_date"] - (times_df["transplant_date"])).dt.days) / 30.4)-15)
    median = times_df['months_since_prev'].median()
    lower_q = times_df['months_since_prev'].quantile(0.25)
    upper_q = times_df['months_since_prev'].quantile(0.75)
    
    demo_dict['CV_EVENTS']['Total'] = {'N':len(times_df),
                                       'Median':median,
                                       'Lower':lower_q, 'Upper':upper_q}   
    demo_dict['CV_EVENTS']['Total']['Arrhythmia'] = processed_events['icd10_code'].str.startswith(tuple(ARYTHMIA_CODES)).sum().sum()
    demo_dict['CV_EVENTS']['Total']['Valvular'] = processed_events['icd10_code'].str.startswith(tuple(VALV_CODES)).sum().sum()
    demo_dict['CV_EVENTS']['Total']['ACS'] = processed_events['icd10_code'].str.startswith(tuple(ACS_CODES)).sum().sum()
    demo_dict['CV_EVENTS']['Total']['CAD'] = processed_events['icd10_code'].str.startswith(tuple(CAD_CODES)).sum().sum()
    demo_dict['CV_EVENTS']['Total']['Cerebrovascular'] = processed_events['icd10_code'].str.startswith(tuple(CEREBRO_CODES)).sum().sum()
    demo_dict['CV_EVENTS']['Total']['Heart failure'] = processed_events['icd10_code'].str.startswith(tuple(HF_CODES)).sum().sum()
    
    print(demo_dict)
    
    def convert_to_native(obj):
        if isinstance(obj, dict):
            return {k: convert_to_native(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_to_native(v) for v in obj]
        elif isinstance(obj, (np.integer, np.int64)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float64)):
            return float(obj)
        elif pd.isna(obj):
            return None
        return obj

    demo_dict_clean = convert_to_native(demo_dict)
    
    # save the cohort info to a json file
    with open(os.path.join(outpath, 'cohort_info.json'), 'w') as f:
        json.dump(demo_dict_clean, f, indent=4)   
        
# run
get_cohort_info(cohort, processed_events, outpath)
# save the wide time series cohort table
cohort.to_csv(os.path.join(outpath, 'preprocessed_cohort_WIDE.csv'), index=False)

{'Females': 0, 'Males': 2, 'Age': {'Median': 40.44, 'Lower': 32.615, 'Upper': 48.265}, 'Current, ex-smokers': 1, 'Indications': {'METAB': 1, 'ALD': 1, 'CANCER': 1, 'HEP': 0, 'FULM': 0, 'IMMUNE': 0, 'RE_TX': 0}, 'DM': {'First': 0, 'Last': 0}, 'HTN': {'First': 0, 'Last': 1}, 'LIP': {'First': 0, 'Last': 0}, 'CV_HISTORY': {'First': 1, 'Last': 2}, 'ANTI_HTN': {'First': 0, 'Last': 1}, 'ANTI_PLATELET': {'First': 0, 'Last': 0}, 'STATIN': {'First': 0, 'Last': 0}, 'ALT': {'Missing, final': 0, 'Median, final': 62.5, 'Lower, final': 46.25, 'Upper, final': 78.75}, 'ALP': {'Missing, final': 0, 'Median, final': 80.0, 'Lower, final': 69.0, 'Upper, final': 91.0}, 'AST': {'Missing, final': 0, 'Median, final': 20.0, 'Lower, final': 19.0, 'Upper, final': 21.0}, 'BMI': {'Missing, final': 0, 'Median, final': 33.1, 'Lower, final': 32.55, 'Upper, final': 33.650000000000006}, 'CREATININE': {'Missing, final': 0, 'Median, final': 77.0, 'Lower, final': 73.0, 'Upper, final': 81.0}, 'CYCLO': {'Missing, final': 0.0,

In [24]:
# reformat the processed data table into format for model predictions
# selecting a single time point from which to make a prediction for each patient

# funcs
def get_prediction_cohort(cohort):
    '''
        For each patient, take the follow up year with the least missingness.
        If there is a tie, take the earlier one.
        Set up the censoring dates
    '''
    pred_cohort = cohort[['person_id', 'transplant_date', 'CENSOR_DATE', 'sex', 'age_at_tx', 'SMOKER', 'METAB', 'ALD', \
                          'CANCER', 'HEP', 'FULM', 'IMMUNE', 'RE_TX']].copy()
    
    max_year = int((pd.to_datetime(STUDY_CUTOFF_DATE) - cohort['transplant_date'].min()).days / 365.25)
    years = list(range(1, max_year + 1))
    
    years_to_censor = (cohort['CENSOR_DATE'] - cohort['transplant_date']).dt.days / 365.25
    
    missingness = pd.DataFrame(0,index=cohort.index, columns=years)
    lab_cols = ['ALT', 'ALP', 'AST', 'BMI', 'CREATININE', 'CYCLO', 'TAC']
    for col in lab_cols:
        for year in years:
            col_name = f'{col}_{year}'
            # fill any value past years_to_censor with nan
            cohort.loc[years_to_censor<year+0.25 ,col_name] = np.nan
            missingness[year] += (cohort[col_name].isnull()).astype(int)
    best_year = missingness.idxmin(axis=1)
    
    selected_values = []

    varying_cols = lab_cols + ['DM', 'HTN', 'LIP', 'CV_HISTORY', 'ANTI_HTN', 'ANTI_PLATELET', 'STATIN', 'MONTHS_TO_EVENT']
    for idx, year in best_year.items():
        patient_values = []
        for c in varying_cols:
            col_name = f'{c}_{year}'
            patient_values.append(cohort.at[idx, col_name])
        selected_values.append(patient_values)
        
    pred_cohort[varying_cols] = pd.DataFrame(selected_values, columns=varying_cols)
    pred_cohort['YRS_SINCE_TRANS'] = best_year + 0.25
    pred_cohort['CURR_AGE'] = pred_cohort['age_at_tx']+pred_cohort['YRS_SINCE_TRANS']
    
    pred_cohort['EVENT'] = pred_cohort['MONTHS_TO_EVENT'].notnull().astype(int)
    # fill null MONTHS_TO_EVENT with end point - anchor date (date of transplant + years since transplant)
    # pred_cohort['anchor_dates'] = pred_cohort.apply(lambda row: row['transplant_date'] + pd.DateOffset(years=row['YRS_SINCE_TRANS'],months=3), axis=1)
    pred_cohort['anchor_dates'] = pred_cohort['transplant_date'] + pd.to_timedelta(
                                        (pred_cohort['YRS_SINCE_TRANS'] * 365.25).round().astype(int), unit="D") 
    pred_cohort['MONTHS_TO_EVENT'] = (pred_cohort['MONTHS_TO_EVENT'].fillna((pred_cohort['CENSOR_DATE'] - pred_cohort['anchor_dates']).dt.days / 30.4)).round()
    pred_cohort.drop(columns = ['transplant_date','CENSOR_DATE','anchor_dates'], inplace=True)
    
    pred_cohort.rename(columns={'age_at_tx':'AGE_AT_TX','person_id':'ID','sex':'SEX', 'CYCLO':'CYCLOSPORINE_TROUGH_LEVEL',
                                'TAC':"TACROLIMUS_TROUGH_LEVEL",'CREATININE' : "SERUM_CREATININE"}, inplace=True)
    
    return pred_cohort

# run
# prediction_cohort = get_prediction_cohort(cohort)


def get_prediction_cohort_random(cohort):
    pred_cohort = cohort[['person_id', 'transplant_date', 'CENSOR_DATE', 'sex', 'age_at_tx', 'SMOKER', 'METAB', 'ALD', \
                          'CANCER', 'HEP', 'FULM', 'IMMUNE', 'RE_TX']].copy()
    
    max_year = int((pd.to_datetime(STUDY_CUTOFF_DATE) - cohort['transplant_date'].min()).days / 365.25)
    years = list(range(1, max_year + 1))
    
    years_to_censor = (cohort['CENSOR_DATE'] - cohort['transplant_date']).dt.days / 365.25
    
    lab_cols = ['ALT', 'ALP', 'AST', 'BMI', 'CREATININE', 'CYCLO', 'TAC']
    for col in lab_cols:
        for year in years:
            col_name = f'{col}_{year}'
            # fill any value past years_to_censor with nan
            cohort.loc[years_to_censor<(year+0.25) ,col_name] = np.nan
    varying_cols = lab_cols + ['DM', 'HTN', 'LIP', 'CV_HISTORY', 'ANTI_HTN', 'ANTI_PLATELET', 'STATIN', 'MONTHS_TO_EVENT']
    random_years=[]
    for idx, censor_time in years_to_censor.items():
        # eligible years = those that are before censor time
        eligible_years = [y for y in years if censor_time >= (y+0.25)]
        if len(eligible_years) == 0:
            random_years.append(np.nan)
        else:
            random_years.append(np.random.choice(eligible_years))   
    random_years = pd.Series(random_years, index=cohort.index, name="random_year")
    orig_len = pred_cohort.shape[0]
    pred_cohort = pred_cohort[~random_years.isna()]
    filtered_len = pred_cohort.shape[0] 
    random_years = random_years.dropna()
    print(f'Dropped {orig_len - filtered_len} patients with no eligible follow-up year for random selection.')
    selected_values = []
    for idx, year in random_years.items():
        patient_values = []
        if pd.isna(year):
            patient_values = [np.nan] * len(varying_cols)
        else:
            for c in varying_cols:
                col_name = f'{c}_{int(year)}'
                val = cohort.at[idx, col_name] if col_name in cohort.columns else np.nan
                patient_values.append(val)
        selected_values.append(patient_values)

    pred_cohort[varying_cols] = pd.DataFrame(selected_values, columns=varying_cols)
    pred_cohort['YRS_SINCE_TRANS'] = random_years + 0.25
    pred_cohort['CURR_AGE'] = pred_cohort['age_at_tx']+pred_cohort['YRS_SINCE_TRANS']
    
    pred_cohort['EVENT'] = pred_cohort['MONTHS_TO_EVENT'].notnull().astype(int)
    # fill null MONTHS_TO_EVENT with end point - anchor date (date of transplant + years since transplant)
    pred_cohort['anchor_dates'] = pred_cohort['transplant_date'] + pd.to_timedelta(
                                        (pred_cohort['YRS_SINCE_TRANS'] * 365.25).round().astype(int), unit="D") #pred_cohort.apply(lambda row: row['transplant_date'] + pd.DateOffset(days = int(row['YRS_SINCE_TRANS']*365.25)), axis=1)
    pred_cohort['MONTHS_TO_EVENT'] = (pred_cohort['MONTHS_TO_EVENT'].fillna((pred_cohort['CENSOR_DATE'] - pred_cohort['anchor_dates']).dt.days / 30.4)).round()
    pred_cohort.drop(columns = ['transplant_date','CENSOR_DATE','anchor_dates'], inplace=True)
    
    pred_cohort.rename(columns={'age_at_tx':'AGE_AT_TX','person_id':'ID','sex':'SEX', 'CYCLO':'CYCLOSPORINE_TROUGH_LEVEL',
                                'TAC':"TACROLIMUS_TROUGH_LEVEL",'CREATININE' : "SERUM_CREATININE"}, inplace=True)
    
    return pred_cohort

prediction_cohort = get_prediction_cohort_random(cohort)



prediction_cohort.to_csv(os.path.join(outpath, 'prediction_cohort.csv'), index=False)
